In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.append('/home/saptarshi/Dhruv/Dissertation/ICC/')
from generate_attributions import *
from utils import CONFIG,evaluate_exp,PREFIX,aggregate_auc_or_sum

## German Credit Dataset

In [2]:
config = CONFIG['german']
mlp_path = PREFIX + 'models/german_mlp/german_mlp_seed_20/logs/version_2/checkpoints/epoch=69-step=910.ckpt'
scm_path = PREFIX + 'models/german_nf/german_nf_seed_20/logs/version_1/checkpoints/epoch=4-step=65.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [3]:
# Load JSON
with open(PREFIX + "attributions/german/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

/home/saptarshi/Dhruv/Dissertation/utils.py:335: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [ ]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'ig': {'pgi': {'k=1': [0.2834436297416687], 'k=2': [0.4305715560913086], 'k=3': [0.4640594720840454], 'k=4': [0.4394669830799103]}, 'pgu': {'k=1': [0.37550804018974304], 'k=2': [0.2160230427980423], 'k=3': [0.0997319445014], 'k=4': [1.3115257324614049e-08]}}, 'itg': {'pgi': {'k=1': [0.2827221751213074], 'k=2': [0.2779334783554077], 'k=3': [0.3328951895236969], 'k=4': [0.43925437331199646]}, 'pgu': {'k=1': [0.37531816959381104], 'k=2': [0.4097268283367157], 'k=3': [0.3625149130821228], 'k=4': [1.3115257324614049e-08]}}, 'sg': {'pgi': {'k=1': [0.2832399010658264], 'k=2': [0.27819880843162537], 'k=3': [0.3329780101776123], 'k=4': [0.4406740963459015]}, 'pgu': {'k=1': [0.37555426359176636], 'k=2': [0.4107031524181366], 'k=3': [0.3623928129673004], 'k=4': [1.3115257324614049e-08]}}, 'shap': {'pgi': {'k=1': [0.28329214453697205], 'k=2': [0.3498416244983673], 'k=3': [0.46495264768600464], 'k=4': [0.4405306577682495]}, 'pgu': {'k=1': [0.3764426112174988], 'k=2': [0.3220416009426117], 'k=3': [

In [8]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


ig: PGI AUC = 0.41869543526321656 +- 0.0038299893377067608, PGU AUC = 0.16783634725492447 +- 0.0014468226968961573
itg: PGI AUC = 0.3239389817168315 +- 0.0024381586441127594, PGU AUC = 0.31996693789338077 +- 0.003621975642783754
sg: PGI AUC = 0.3243779438609879 +- 0.0024606683927141547, PGU AUC = 0.3202910452025632 +- 0.0036036721828803095
shap: PGI AUC = 0.39223521761596203 +- 0.003536491473283097, PGU AUC = 0.2033945016392196 +- 0.001337547702222217
lime: PGI AUC = 0.32388957787305117 +- 0.002465682567665392, PGU AUC = 0.3206153471395373 +- 0.0036394226865942504
sp_lime: PGI AUC = 0.2870905180772146 +- 0.0020504536041592563, PGU AUC = 0.31529172834319374 +- 0.002790432489363055
pfi: PGI AUC = 0.4002129001791278 +- 0.0033153579896532167, PGU AUC = 0.20425954149415093 +- 0.0023715669385674367
icc_topo: PGI AUC = 0.3997403827185433 +- 0.0032562497949439975, PGU AUC = 0.20416983026079832 +- 0.0023567664769416096
icc_shap: PGI AUC = 0.3996318925420443 +- 0.003323589853585961, PGU AUC = 0.

In [9]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

ig: PGI SUM = 1.617541790008545 +- 0.01430930182604274, PGU SUM = 0.6912630200386047 +- 0.004964896647240891
itg: PGI SUM = 1.3328051567077637 +- 0.010188912119126758, PGU SUM = 1.1475598812103271 +- 0.01267977716839483
sg: PGI SUM = 1.3350907564163208 +- 0.010267774141046706, PGU SUM = 1.1486502885818481 +- 0.012613822618831344
shap: PGI SUM = 1.5386171340942383 +- 0.013594305829164528, PGU SUM = 0.79840487241745 +- 0.005564039386932357
lime: PGI SUM = 1.3326388597488403 +- 0.010307256255060498, PGU SUM = 1.1498894691467285 +- 0.012723786907609906
sp_lime: PGI SUM = 1.1311869621276855 +- 0.007442872136563118, PGU SUM = 1.1782479286193848 +- 0.010600956568908663
pfi: PGI SUM = 1.5624566078186035 +- 0.012796003425190897, PGU SUM = 0.8006547689437866 +- 0.008280316318748817
icc_topo: PGI SUM = 1.5606731176376343 +- 0.012563240078590995, PGU SUM = 0.800071120262146 +- 0.008216354261448684
icc_shap: PGI SUM = 1.5601816177368164 +- 0.012806962659474942, PGU SUM = 0.8012599945068359 +- 0.008

## Lung Cancer Dataset

In [10]:
config = CONFIG['cancer']
mlp_path = PREFIX + 'models/cancer_mlp/cancer_mlp_seed_20/logs/version_1/checkpoints/epoch=62-step=2394.ckpt'
scm_path = PREFIX + 'models/cancer_nf/cancer_nf_seed_30/logs/version_1/checkpoints/epoch=143-step=5472.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [11]:
# Load JSON
with open(PREFIX +"attributions/cancer/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

/home/saptarshi/Dhruv/Dissertation/utils.py:335: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



In [12]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'ig': {'pgi': {'k=1': [0.2913896143436432], 'k=2': [0.39338016510009766], 'k=3': [0.4128062427043915], 'k=4': [0.4324224293231964], 'k=5': [0.4575633108615875], 'k=6': [0.45971548557281494], 'k=7': [0.46695128083229065]}, 'pgu': {'k=1': [0.4169473648071289], 'k=2': [0.37334969639778137], 'k=3': [0.33514103293418884], 'k=4': [0.26316919922828674], 'k=5': [0.18634921312332153], 'k=6': [0.13755084574222565], 'k=7': [1.802997395827788e-08]}}, 'itg': {'pgi': {'k=1': [0.21413284540176392], 'k=2': [0.2690756618976593], 'k=3': [0.3100980520248413], 'k=4': [0.3636414706707001], 'k=5': [0.38608819246292114], 'k=6': [0.417403906583786], 'k=7': [0.46694889664649963]}, 'pgu': {'k=1': [0.43740397691726685], 'k=2': [0.41934576630592346], 'k=3': [0.39960595965385437], 'k=4': [0.365527868270874], 'k=5': [0.3584193289279938], 'k=6': [0.291490375995636], 'k=7': [1.802997395827788e-08]}}, 'sg': {'pgi': {'k=1': [0.21405939757823944], 'k=2': [0.2986878752708435], 'k=3': [0.3286169767379761], 'k=4': [0.3507

In [13]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


ig: PGI AUC = 0.4225096924654726 +- 0.005488272159046982, PGU AUC = 0.25067228648682227 +- 0.0032964734576961673
itg: PGI AUC = 0.3478080348147907 +- 0.005456273520666611, PGU AUC = 0.3421818839815549 +- 0.003779714737162966
sg: PGI AUC = 0.3533844070876431 +- 0.005371129768377791, PGU AUC = 0.33727048266303933 +- 0.004119133812813169
shap: PGI AUC = 0.41688198690613115 +- 0.005468324732260099, PGU AUC = 0.2695781432336015 +- 0.0037620674791127287
lime: PGI AUC = 0.39002131824329905 +- 0.004409837510022869, PGU AUC = 0.301142644093359 +- 0.004707749807535575
sp_lime: PGI AUC = 0.3480284837057762 +- 0.004911049944984952, PGU AUC = 0.3104811566410353 +- 0.0037995809792164474
pfi: PGI AUC = 0.41605347548197547 +- 0.005132366837483357, PGU AUC = 0.2772527312803626 +- 0.00427546581426696
icc_topo: PGI AUC = 0.4219518378376961 +- 0.005288275375924724, PGU AUC = 0.263133323189298 +- 0.003899615785070727
icc_shap: PGI AUC = 0.41338039264289866 +- 0.005035465954676548, PGU AUC = 0.2768736083244

In [14]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

ig: PGI SUM = 2.9142284393310547 +- 0.037133408755837705, PGU SUM = 1.7125074863433838 +- 0.022920448317326696
itg: PGI SUM = 2.427389144897461 +- 0.03774930896420796, PGU SUM = 2.2717931270599365 +- 0.02571348329713967
sg: PGI SUM = 2.461150646209717 +- 0.037263714617992306, PGU SUM = 2.2427613735198975 +- 0.027751267446584113
shap: PGI SUM = 2.841600179672241 +- 0.03780284751843999, PGU SUM = 1.8363761901855469 +- 0.025530468652909793
lime: PGI SUM = 2.7195284366607666 +- 0.03069511655757576, PGU SUM = 2.0156853199005127 +- 0.03153249293131904
sp_lime: PGI SUM = 2.3837890625 +- 0.03351636379071871, PGU SUM = 2.0912580490112305 +- 0.025934297591466825
pfi: PGI SUM = 2.875366687774658 +- 0.034996884153087764, PGU SUM = 1.8725628852844238 +- 0.028866924094871974
icc_topo: PGI SUM = 2.910930871963501 +- 0.03592795178006238, PGU SUM = 1.787812352180481 +- 0.026594436092534944
icc_shap: PGI SUM = 2.8596153259277344 +- 0.03443336422461928, PGU SUM = 1.8699170351028442 +- 0.02924203707676632

## Synthetic Data

In [15]:
config = CONFIG['syn']
mlp_path = PREFIX + 'models/syn_mlp/syn_mlp_seed_20/logs/version_3/checkpoints/epoch=12-step=416.ckpt'
scm_path = PREFIX + 'models/syn_nf/syn_nf_seed_20/logs/version_1/checkpoints/epoch=3-step=128.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [16]:
# Load JSON
with open(PREFIX +"attributions/syn/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

/home/saptarshi/Dhruv/Dissertation/utils.py:335: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



In [17]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'ig': {'pgi': {'k=1': [0.029641829431056976], 'k=2': [0.04707638919353485], 'k=3': [0.06605358421802521]}, 'pgu': {'k=1': [0.04477120563387871], 'k=2': [0.024550244212150574], 'k=3': [1.8278758062706402e-08]}}, 'itg': {'pgi': {'k=1': [0.02530992403626442], 'k=2': [0.04489035904407501], 'k=3': [0.06598171591758728]}, 'pgu': {'k=1': [0.04945721477270126], 'k=2': [0.029733307659626007], 'k=3': [1.8278758062706402e-08]}}, 'sg': {'pgi': {'k=1': [0.029758138582110405], 'k=2': [0.04700464382767677], 'k=3': [0.06600736081600189]}, 'pgu': {'k=1': [0.0449242927134037], 'k=2': [0.024578919634222984], 'k=3': [1.8278758062706402e-08]}}, 'shap': {'pgi': {'k=1': [0.02976806089282036], 'k=2': [0.04719151183962822], 'k=3': [0.06590164452791214]}, 'pgu': {'k=1': [0.04495255649089813], 'k=2': [0.024512970820069313], 'k=3': [1.8278758062706402e-08]}}, 'lime': {'pgi': {'k=1': [0.02980726957321167], 'k=2': [0.04713594540953636], 'k=3': [0.06594235450029373]}, 'pgu': {'k=1': [0.04491646960377693], 'k=2': [0

In [18]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


ig: PGI AUC = 0.047462050961330535 +- 0.00014003795270138057, PGU AUC = 0.02346792757627554 +- 0.00011258250300568364
itg: PGI AUC = 0.04526808951360484 +- 0.00016655370995491823, PGU AUC = 0.027230962325508395 +- 7.72468179971054e-05
sg: PGI AUC = 0.04744369451887906 +- 0.0001356101715283818, PGU AUC = 0.023520537379663437 +- 0.00011399586525231044
shap: PGI AUC = 0.04751318217255175 +- 0.0001378855023179437, PGU AUC = 0.023494629290653393 +- 0.00011578844011125173
lime: PGI AUC = 0.04750537770800292 +- 0.0001390707334223651, PGU AUC = 0.02353385152799698 +- 0.00011239355886928488
sp_lime: PGI AUC = 0.047311777852786084 +- 0.00014663119885256816, PGU AUC = 0.024401637860573828 +- 0.00010340821818564929
pfi: PGI AUC = 0.04857908478006721 +- 0.0001317760848468028, PGU AUC = 0.023863628326604765 +- 0.00010964727683924675
icc_topo: PGI AUC = 0.047508014403283595 +- 0.0001404679927304095, PGU AUC = 0.02352826683083549 +- 0.00011278846185856647
icc_shap: PGI AUC = 0.04744294467071692 +- 0.0

In [19]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

ig: PGI SUM = 0.14277181029319763 +- 0.00040685526483039337, PGU SUM = 0.06932146847248077 +- 0.0003121837329777222
itg: PGI SUM = 0.1361819952726364 +- 0.0004860816624607593, PGU SUM = 0.07919053733348846 +- 0.00022333583366690075
sg: PGI SUM = 0.14277012646198273 +- 0.00039645528143669804, PGU SUM = 0.06950322538614273 +- 0.00031574436955712125
shap: PGI SUM = 0.14286120235919952 +- 0.00040222612170217, PGU SUM = 0.06946554780006409 +- 0.00032297584683048955
lime: PGI SUM = 0.14288556575775146 +- 0.000405479776042121, PGU SUM = 0.06952594965696335 +- 0.0003127900552185542
sp_lime: PGI SUM = 0.1398528665304184 +- 0.0004502077422923579, PGU SUM = 0.07238444685935974 +- 0.0002827668386044482
pfi: PGI SUM = 0.1450340449810028 +- 0.00039063014706199495, PGU SUM = 0.07023367285728455 +- 0.00030956069635185157
icc_topo: PGI SUM = 0.14285241067409515 +- 0.0004099148215319517, PGU SUM = 0.06952177733182907 +- 0.0003128755836490271
icc_shap: PGI SUM = 0.14270542562007904 +- 0.00039994398212347

## Test

In [2]:
config = CONFIG['syn']
mlp_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_ann/syn_ann_seed_10/logs/version_0/checkpoints/epoch=161-step=5184.ckpt'
scm_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_nf/syn_nf_seed_10/logs/version_0/checkpoints/epoch=19-step=640.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_ann'],classification=False))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))
ge_dict, evaluation_metrics, time_dict, global_explanations = generate_global_exps(config,mlp_model,scm_model)

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [3]:
ge_dict

{'ig': {'W': [0.008685602410414437],
  'Z': [0.5541948194138167],
  'X': [0.4371195781757688]},
 'itg': {'W': [0.008485306748260027],
  'Z': [0.497316907775009],
  'X': [0.49419778547673104]},
 'sg': {'W': [0.13136424456458626],
  'Z': [0.3860659244300918],
  'X': [0.48256983100532197]},
 'shap': {'W': [0.5803365821763931],
  'Z': [0.3543153621348831],
  'X': [0.06534805568872387]},
 'lime': {'W': [0.2837432524861816],
  'Z': [0.293367905107146],
  'X': [0.4228888424066725]},
 'sp_lime': {'X': [0.41602805243460733],
  'W': [0.2947343644233361],
  'Z': [0.2892375831420565]},
 'pfi': {'W': [0.2555289566516876],
  'Z': [0.2728649377822876],
  'X': [0.47160604596138]},
 'icc_topo': {'W': [0.07992755621671677],
  'Z': [0.6083893179893494],
  'X': [0.31013962626457214]},
 'icc_shap': {'W': [0.1287594437599182],
  'Z': [0.6630550622940063],
  'X': [0.20664207637310028]}}

In [9]:
generate_attr_plot(global_explanations,config)

In [7]:
# Load JSON
with open("/home/dhruv/Files/Thesis/Dissertation/Code/attributions/syn/attribution.json", "r") as f:
    data = json.load(f)

# Call the function
summary = compute_pgu_pgi_sums(data["evaluation_metrics"])

# Display the results
for method, scores in summary.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")


ig: PGU Sum = 0.128419, PGI Sum = 0.000436
itg: PGU Sum = 0.128419, PGI Sum = 0.000436
sg: PGU Sum = 0.128418, PGI Sum = 0.000437
shap: PGU Sum = 0.128420, PGI Sum = 0.000435
lime: PGU Sum = 0.128418, PGI Sum = 0.000437
sp_lime: PGU Sum = 0.128420, PGI Sum = 0.000435
pfi: PGU Sum = 0.128418, PGI Sum = 0.000437
icc_topo: PGU Sum = 0.128419, PGI Sum = 0.000436
icc_shap: PGU Sum = 0.128419, PGI Sum = 0.000436
